In [2]:
# Gather the correct libraries 
import os 
import csv 
import requests 
import time
from bs4 import BeautifulSoup
import pandas as pd


In [4]:
# Define the base URL for the boxscores
base_url = 'https://bsnpr.com/juegos/juego?r='

# Define a list of team url number, team name and team town

teams = [
    (1, 'Cangrejeros', 'Santurce'),
    (2, 'Capitanes', 'Arecibo'),
    (3, 'Atleticos', 'San German'),
    (5, 'Leones', 'Ponce'),
    (6, 'Piratas', 'Quebradillas'),
    (9, 'Vaqueros', 'Bayamon'),
    (10, 'Gigantes', 'Carolina'),
    (12, 'Indios', 'Mayaguez'),
    (14, 'Mets', 'Guaynabo'),
    (15, 'Cariduros', 'Fajardo'),
    (73, 'Grises', 'Humacao'),
    (79, 'Osos', 'Manati')
]

# Define the range of years, months, and days to scrape
years = range(2013, 2024)
months = range(10, 13)
days = range(1, 100)

# Loop over the years, months, and days
for year in years:
    # Create a folder for the year if it doesn't exist
    if not os.path.exists(str(year)):
        os.makedirs(str(year))
    for month in months:
         for day in days:
            # Construct the date string 
            date_str = f"{year}{month:02d}{day:02d}"
            #f"{year}{month:02d}{day:02d}"
            url_bs = base_url + 'BS' + date_str
            
            while True:
                # Send a request to the URL and parse the HTML
                response = requests.get(url_bs)
                if response.status_code == 200:
                    print(f'Processing date: {date_str}')
                    
                    soup = BeautifulSoup(response.content, 'html.parser')
                
                    tables = soup.findAll("table", class_="tabla_stats")
                    if len(tables) < 2:
                        print(f"No boxscore found for date {date_str}")
                        break
                    else:
                        t_boxscore = tables[1]
                        t_hometeam = tables[2]
                        t_awayteam = tables[3]
                    
                    # Extract the team names and scores from the table_bs
                
                    team_towns = []
                    team_scores = []
                    for row in t_boxscore.findAll("tr")[1:]:
                        cols = row.findAll("td")
                        team_towns.append(cols[0].text.strip())
                        team_scores.append(cols[-1].text.strip())
                
                    # Extract the table data
                    final_boxscore = []
                    for row in t_boxscore.findAll("tr")[1:]:
                        cols = row.findAll("td")[1:-1]
                        row_data = [col.text.strip() for col in cols]
                        final_boxscore.append(row_data)
                    df_table = pd.DataFrame(final_boxscore[:4], columns=['1Q', '2Q', '3Q', '4Q'])
                    # Create a DataFrame from the table data 
                    ## Check if the boxscore has at least 4 quarters
                    if len(final_boxscore) >= 4:
                        # Create a DataFrame from the table data with 4 columns for the first 4 quarters
                        
    
                        # If there are more than 4 quarters, add an 'OT' column for each additional quarter
                        for i in range(4, len(final_boxscore)):
                                column_name = f"OT{i-3}"  # Name the column 'OT5' for the 5th quarter, 'OT6' for the 6th, and so
                                df_table[column_name] = final_boxscore[i]
                            
                            # Update the game_id variable
                                game_id += f"_OT{i-3}"

                    # Create a DataFrame for the team names and scores 
                                df_team = pd.DataFrame({'Team': team_towns, 'Score': team_scores})

                    # Concatenate the two DataFrames
                                df = pd.concat([df_team, df_table], axis=1)
                                df.drop(2, inplace=True)
                                print(df.iloc[0]['Team'], "vs", df.iloc[1]['Team'])
                    
                    # Save the box score file in a yearly folder with the date_str as game_id variable
                                folder_path = f"/Users/jeanzayas/Desktop/Divergence/Portfolio/Sports Analysis/Basketball/BSN/dataset/{year}"
                                if not os.path.exists(folder_path):
                                    os.makedirs(folder_path)
                                    game_id = date_str
                                file_path = f"{folder_path}/{game_id}.csv"
                                df.to_csv(file_path, index=False)
                    
                    # Extract the table data fro the home team
                                team_names = []
                                team_scores = []
                                for row in t_boxscore.findAll("tr")[1:]:
                                    cols = row.findAll("td")
                                team_names.append(cols[0].text.strip())
                                team_scores.append(cols[-1].text.strip())
    
                    # Extract the table data
                                table_data = []
                                for row in t_boxscore.findAll("tr")[1:]:
                                    cols = row.findAll("td")[1:-1]
                                    row_data = [col.text.strip() for col in cols]
                                table_data.append(row_data)

                    ## Check if the boxscore has at least 4 quarters
                                if len(table_data) >= 4:
                        # Create a DataFrame from the table data with 4 columns for the first 4 quarters
                                    df_table = pd.DataFrame(table_data[:4], columns=['1Q', '2Q', '3Q', '4Q'])
    
                        # If there are more than 4 quarters, add an 'OT' column for each additional quarter
                                for i in range(4, len(table_data)):
                                    column_name = f"OT{i-3}"  # Name the column 'OT5' for the 5th quarter, 'OT6' for the 6th, and so
                                    df_table[column_name] = table_data[i]
                                else:
    # Add the boxscore to the list of errors if it doesn't have at least 4 quarters
                    

    
                    # Save to CSV
                                    df_home.to_csv(f"/Users/jeanzayas/Desktop/Divergence/Portfolio/Sports Analysis/Basketball/BSN/dataset/{year}/{date_str}_home.csv", index=False)
                                    df_away.to_csv(f"/Users/jeanzayas/Desktop/Divergence/Portfolio/Sports Analysis/Basketball/BSN/dataset/{year}/{date_str}_away.csv", index=False)
                    
                    # Exit the while loop
                                    break
            
 
                                    print(f'No data found for BoxScore: {date_str}')
                                    continue

Processing date: 20131001
Processing date: 20131001
Processing date: 20131001
Processing date: 20131001


KeyboardInterrupt: 